In [ ]:
import os
import json
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.2 MB 5.2 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 281 kB 69.4 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
     |████████████████████████████████| 103 kB 76.8 MB/s 
     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 109 kB 64.9 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 22.7 MB 1.0 MB/s 
     |████████████████████████████████| 11.1 MB 22.3 MB/s 
     |████████████████████████████████| 51 kB 858 kB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 3.2 MB 4.2 MB/s 
     |████████████████████████████████| 546 kB 62.3 MB/s 
     |████████████████████████████████| 90 kB

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from dgl import DGLGraph
import re
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
ratings = []
with open(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/인턴/ml-1m/', 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
        })
ratings = pd.DataFrame(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
len(ratings.index)

1000209

In [ ]:
nodeidx = {}
ratings['user_id'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

In [ ]:
min(list(ratings['movie_id'].unique()))

1

In [ ]:
max(list(ratings['movie_id'].unique()))

3952

In [ ]:
np_movieid = ratings['movie_id'].values
np_movieid

array([1193,  661,  914, ...,  562, 1096, 1097])

In [ ]:
npplus = np.full((1000209,),6040)

In [ ]:
#min(list(np_movieid + npplus )),max(list(np_movieid + npplus ))

In [ ]:
ratings['movie_id_plus'] = list(np_movieid +  npplus)
ratings.head()

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395


In [ ]:
'''
인덱스를 따로 부여할때 
idx = 0 
uniuser = ratings['user_id'].unique()
unimovie = ratings['movie_id_plus'].unique()

for i in range(len(uniuser)):
  nodeidx[idx] = uniuser[i]
  idx += 1

for i in range(len(unimovie)):
  nodeidx[idx] = unimovie[i]
  idx += 1

nodeidx'''

"\n인덱스를 따로 부여할때 \nidx = 0 \nuniuser = ratings['user_id'].unique()\nunimovie = ratings['movie_id_plus'].unique()\n\nfor i in range(len(uniuser)):\n  nodeidx[idx] = uniuser[i]\n  idx += 1\n\nfor i in range(len(unimovie)):\n  nodeidx[idx] = unimovie[i]\n  idx += 1\n\nnodeidx"

In [ ]:
nodeidx = {}
for i in range(9993):
  nodeidx[i] = i

In [ ]:
ratings.head(10)

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395
5,1,1197,3,978302268,7237
6,1,1287,5,978302039,7327
7,1,2804,5,978300719,8844
8,1,594,4,978302268,6634
9,1,919,4,978301368,6959


In [ ]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=100000000

In [ ]:
users = []
movies = []

for i in range(len(ratings.index)):
  user = ratings.iloc[i,0]
  users.append(nodeidx[user])

  movie = ratings.iloc[i, 4]
  movies.append(nodeidx[movie])


In [ ]:
edges_src = torch.from_numpy(np.array(users))
edges_dst = torch.from_numpy(np.array(movies))

In [ ]:
graph = dgl.graph((edges_src, edges_dst))
print(graph)

Graph(num_nodes=9993, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
edge_features = torch.from_numpy(ratings['rating'].to_numpy())
len(edge_features)

1000209

In [ ]:
graph.edata['w'] = edge_features

In [ ]:
print(graph)

Graph(num_nodes=9993, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.int64)})


https://docs.dgl.ai/tutorials/blitz/4_link_predict.html?highlight=negative

In [ ]:
u, v = graph.edges()

In [ ]:
eids = np.arange(graph.number_of_edges()) #엣지를 가지고 train, test로 나누어줌

eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)    #엣지 중 10%를 택함
train_size = graph.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [ ]:
len(eids)

1000209

In [ ]:
len(u.numpy())

1000209

In [ ]:
len(u)

1000209

In [ ]:
len(np.ones(len(u)))

1000209

In [ ]:
len(u.numpy()), len(v.numpy())

(1000209, 1000209)

In [ ]:
graph.num_nodes()

9993

In [ ]:
eids = np.arange(graph.number_of_edges())  #[0, 1, 2 .... number_of_edges()]
eids

array([      0,       1,       2, ..., 1000206, 1000207, 1000208])

In [ ]:
eids = np.random.permutation(eids)
eids

array([154462, 694219, 334478, ..., 896429,  83745, 684757])

In [ ]:
len(eids)

1000209

In [ ]:
test_size = int(len(eids) * 0.1)

In [ ]:
train_size = graph.number_of_edges() - test_size

In [ ]:
u, v = graph.edges()

In [ ]:
len(u)

1000209

In [ ]:
v

tensor([7233, 6701, 6954,  ..., 6602, 7136, 7137])

In [ ]:
u.numpy()

array([   1,    1,    1, ..., 6040, 6040, 6040])

In [ ]:
#여기서부터 node pairs with no edges between them) as negative examples.
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())), shape=(9993,9993)      )
print(adj.shape)
adj_neg = 1 - adj.todense() - np.eye(graph.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

(9993, 9993)


In [ ]:
neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [ ]:
train_g = dgl.remove_edges(graph, eids[:test_size])

In [ ]:

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())


In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.
        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.
        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [ ]:
model = GraphSAGE(9993, 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
import itertools
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

In [ ]:
import torch as th

In [ ]:
np.eye(9993, dtype=np.double)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
feat = th.zeros(9993, 9993) + torch.eye(9993)
feat

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, feat)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.693209707736969
In epoch 5, loss: 0.6443096995353699
In epoch 10, loss: 0.5857884883880615
In epoch 15, loss: 0.5723260045051575
In epoch 20, loss: 0.5645340085029602
In epoch 25, loss: 0.5600878000259399
In epoch 30, loss: 0.5568821430206299
In epoch 35, loss: 0.5549263954162598
In epoch 40, loss: 0.5532799959182739
In epoch 45, loss: 0.5512085556983948
In epoch 50, loss: 0.5479452013969421
In epoch 55, loss: 0.5426159501075745
In epoch 60, loss: 0.5366511344909668
In epoch 65, loss: 0.531111478805542
In epoch 70, loss: 0.5264434218406677
In epoch 75, loss: 0.5227557420730591
In epoch 80, loss: 0.5195362567901611
In epoch 85, loss: 0.5169728398323059
In epoch 90, loss: 0.514940619468689
In epoch 95, loss: 0.513027012348175


In [ ]:
h.shape

torch.Size([9993, 16])

In [ ]:
movies = []
with open(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/인턴/ml-1m/', 'movies.dat'), encoding='latin1') as f:
    for l in f:
        id_, title, genres = l.strip().split('::')
        genres_set = set(genres.split('|'))

        # extract year
        assert re.match(r'.*\([0-9]{4}\)$', title)
        year = title[-5:-1]
        title = title[:-6].strip()

        data = {'movie_id': int(id_), 'title': title, 'year': year}
        for g in genres_set:
            data[g] = True
        movies.append(data)
movies = pd.DataFrame(movies).astype({'year': 'category'})

In [ ]:
h[9743]

tensor([ 1.4682,  1.6676,  0.5085,  0.0804,  0.7296, -0.7524,  0.1092, -0.4404,
         0.4374,  0.0999, -0.7759, -0.1496, -0.7970, -0.0778, -1.9149, -0.0214],
       grad_fn=<SelectBackward0>)

In [ ]:
# 벡터 유사도 측정 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
cos_sim(h[9743].detach().numpy(),h[9742].detach().numpy() )

0.9926201

In [ ]:
cos_sim(h[9743].detach().numpy(),h[9744].detach().numpy() )

0.62154704

In [ ]:
cos_sim(h[9743].detach().numpy(),h[7648].detach().numpy() )

0.90922135

In [ ]:
cos_sim(h[9743].detach().numpy(),h[7761].detach().numpy() )

0.9909096

In [ ]:
cos_sim(h[9743].detach().numpy(),h[6151].detach().numpy() )

0.9226807

In [ ]:
cos_sim(h[9743].detach().numpy(),h[6255].detach().numpy() )

-0.03298556

In [ ]:
cos_sim(h[9].detach().numpy(), h[7890].detach().numpy() )

-0.4996043